Converting Columns To Appropraite Data Type with Pandas

In [1]:
# Learning how to convert from different date foramts to another date format

import pandas as pd
from datetime import datetime

# Create the DataFrame
dfs = pd.DataFrame([
    ['tv', 'p232', '19-1-2026', 56, 60000],
    ['chair', 'p232', '19-1-2026', 56, 60000],
    ['lapi', 'p232', '19-11-2026', 56, 60000],
    ['stool', 'p232', '19-1-2026', 56, 60000]
], columns=['p_name', 'p_id', 'sales_date', 'quant', 'price'])

# Convert 'sales_date' to datetime with a specified format
dfs['sales_date'] = pd.to_datetime(dfs['sales_date'], format='%d-%m-%Y')
# take note: '%d-%m-%Y' has to be in the same format with saes_date. If sales_date was for example, 19/1/2026, 
# then you have to use '%d/%m/%Y'.

# Check the DataFrame to see the conversion
print(dfs['sales_date'].head())


0   2026-01-19
1   2026-01-19
2   2026-11-19
3   2026-01-19
Name: sales_date, dtype: datetime64[ns]


In [2]:
# Creating a file with missing values
import pandas as pd
import numpy as np
from random import randint, random
from datetime import date 

def create_sales_data_with_missing_values(file_name):
    header_row = ['product_id', 'customer_id', 'customer_age', 'sales_date', 'quantity', 'price']
    data_rows = []
    
    for i in range(65, 91):  # for 26 letters to make it 26 products
        
        # Randomly introduce missing values (with about 10% chance for each field)
        product_id = f'Product {chr(i)}' 
        customer_id = f'CUST {(i+2)%70 + 1}' 
        sales_date = date(2026, 1, (i-60)).strftime("%d-%m-%Y") # I changed the format of date from "%Y%m%d" to "%d%m%Y" and did not make it part of themissing values because I don't have strength to be filling in empty spaces
        quantity = f'{randint(10, 200)}' if random() > 0.1 else np.NaN
        price = f'{randint(500,10000)}' if random() > 0.1 else np.NaN
        customer_age = f'{randint(18,90)}' if random() > 0.1 else np.NaN

        details = [product_id, customer_id, customer_age, sales_date, quantity, price]
        data_rows.append(details)

    
    df = pd.DataFrame(data_rows, columns=header_row)
    df.to_csv(file_name, index=False)
    print(f"\n{file_name} creation done.")

create_sales_data_with_missing_values('sales_data_5.csv')


sales_data_5.csv creation done.


In [3]:
# 1 - Confirming I have Pandas Installed

import pandas as pd
print(pd.__version__)

2.2.2


In [4]:
# 2 - Read the CSV File

df = pd.read_csv('sales_data_5.csv')
df.info() # checking the datatypes the columns have

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    26 non-null     object 
 1   customer_id   26 non-null     object 
 2   customer_age  23 non-null     float64
 3   sales_date    26 non-null     object 
 4   quantity      24 non-null     float64
 5   price         23 non-null     float64
dtypes: float64(3), object(3)
memory usage: 1.3+ KB


In [5]:
df.head() # checking the format of the date in sales_date column

,product_id,customer_id,customer_age,sales_date,quantity,price
0,Product A,CUST 68,84.0,05-01-2026,133.0,4408.0
1,Product B,CUST 69,81.0,06-01-2026,56.0,9769.0
2,Product C,CUST 70,NaN,07-01-2026,54.0,NaN
3,Product D,CUST 1,NaN,08-01-2026,88.0,795.0
4,Product E,CUST 2,29.0,09-01-2026,44.0,1455.0


In [6]:
# 3 - Convert 'sales_date' to Datetime

from datetime import datetime

df['sales_date'] = pd.to_datetime(df['sales_date'], format='%d-%m-%Y', errors='coerce')

print(df['sales_date'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 26 entries, 0 to 25
Series name: sales_date
Non-Null Count  Dtype         
--------------  -----         
26 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 340.0 bytes
None


In [7]:
# 4 - Convert Numeric Columns

numeric_columns = ['price', 'quantity', 'customer_age']

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(df[numeric_columns].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         23 non-null     float64
 1   quantity      24 non-null     float64
 2   customer_age  23 non-null     float64
dtypes: float64(3)
memory usage: 756.0 bytes
None


In [8]:
print(df[['customer_age', 'price', 'quantity']].isnull().sum())

customer_age    3
price           3
quantity        2
dtype: int64


In [9]:
# 6 - Handle Missing Values

# Always inspect missing values BEFORE imputing
print("Missing values before:\n", df[['price', 'quantity', 'customer_age']].isna().sum())

# Impute numeric columns
# - price: mean (common default)
# - quantity: mode if it exists, otherwise median fallback
# - customer_age: median

df['price'] = df['price'].fillna(round(df['price'].mean(), 0))

m = df['quantity'].mode(dropna=True)
fill_qty = m.iloc[0] if not m.empty else df['quantity'].median()
df['quantity'] = df['quantity'].fillna(fill_qty)

df['customer_age'] = df['customer_age'].fillna(df['customer_age'].median())

print("\nMissing values after:\n", df[['price', 'quantity', 'customer_age']].isna().sum())

Missing values before:
 price           3
quantity        2
customer_age    3
dtype: int64

Missing values after:
 price           0
quantity        0
customer_age    0
dtype: int64


In [11]:
# Using apply yo convert from float to int
df[['price', 'quantity', 'customer_age']] = df[['price', 'quantity', 'customer_age']].apply(pd.to_numeric, errors='coerce').astype('Int64')

print(df[['price', 'quantity', 'customer_age']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   price         26 non-null     Int64
 1   quantity      26 non-null     Int64
 2   customer_age  26 non-null     Int64
dtypes: Int64(3)
memory usage: 834.0 bytes
None


In [12]:
# 7 - Save the Cleaned Data

df.to_csv('cleaned_sales_data_2.csv', index=False)